In [ ]:
"""
This notebook extracts the Wing Height from the live bee images by registering and rotating them. 
"""

In [18]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from glob import glob
from scipy import ndimage
from utils import place_in_img

from array2gif import write_gif

In [20]:
DEBUG = False

mask_fps = glob('../2_live_bees/9_UNet_predictions_resized_final/*')

out = []
for mask_fp in tqdm(mask_fps):
    fn = mask_fp.split('/')[-1]
    if DEBUG:
        fn = '2024_06_06_h05bee16.png'

    seg_fp = '../2_live_bees/9_UNet_predictions_resized_final_overlaid/' + fn
    
    mask = cv2.imread(mask_fp, cv2.IMREAD_GRAYSCALE)
    seg = cv2.imread(seg_fp)
    
    blank_image = np.ones((500,1000,3)).astype('uint8')*255
    
    if DEBUG:
        plt.imshow(blank_image)
        plt.figure()
        plt.imshow(mask)
    
    ay, ax = np.array(np.where(mask == 4)).mean(axis=1)
    by, bx = np.array(np.where(mask == 5)).mean(axis=1)
    ab_dist = np.sqrt((ax-bx)**2 + (ay-by)**2)
    ab_angle = np.arctan((by-ay)/(bx-ax))*180/np.pi
    
    scale = 100 / ab_dist
    rotation_angle = ab_angle
    
    new_shape = (np.array(mask.shape[:2])*scale).astype('int')
    new_mask = cv2.resize(mask, new_shape[::-1])
    new_seg = cv2.resize(seg, new_shape[::-1])
    
    new_mask = ndimage.rotate(new_mask, rotation_angle, mode='constant', cval=0)
    new_seg = ndimage.rotate(new_seg, rotation_angle, mode='constant', cval=255)

    if DEBUG:
        plt.imshow(new_seg)
    
    nay, nax = np.array(np.where(new_mask == 4)).mean(axis=1)
    nay, nax = int(nay), int(nax)
    ncy, ncx = new_mask.shape[0]//2, new_mask.shape[1]//2
    blank_image = place_in_img(blank_image, new_seg, (150+ncy-nay,500+ncx-nax))
    if DEBUG:
        plt.imshow(blank_image)

    blank_image = cv2.cvtColor(blank_image, cv2.COLOR_BGR2RGB)

    y_min = np.where(blank_image < 250)[0].min()
    y_max = np.where(blank_image < 250)[0].max()
    height = y_max - y_min

    out += [[fn, height]]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1182/1182 [04:16<00:00,  4.61it/s]


In [21]:
pd.DataFrame(out, columns=['Filename','Height (pixels)']).to_csv('live_bees_height_pixels.csv')